In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import torch 
from torch import optim, cuda
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vrpbpart2/resized/Horses/45.jpg
/kaggle/input/vrpbpart2/resized/Horses/56.jpg
/kaggle/input/vrpbpart2/resized/Horses/89.jpg
/kaggle/input/vrpbpart2/resized/Horses/20.jpg
/kaggle/input/vrpbpart2/resized/Horses/58.jpg
/kaggle/input/vrpbpart2/resized/Horses/6.jpg
/kaggle/input/vrpbpart2/resized/Horses/76.jpg
/kaggle/input/vrpbpart2/resized/Horses/71.jpg
/kaggle/input/vrpbpart2/resized/Horses/5.jpg
/kaggle/input/vrpbpart2/resized/Horses/8.jpg
/kaggle/input/vrpbpart2/resized/Horses/84.jpg
/kaggle/input/vrpbpart2/resized/Horses/85.jpg
/kaggle/input/vrpbpart2/resized/Horses/67.jpg
/kaggle/input/vrpbpart2/resized/Horses/82.jpg
/kaggle/input/vrpbpart2/resized/Horses/30.jpg
/kaggle/input/vrpbpart2/resized/Horses/97.jpg
/kaggle/input/vrpbpart2/resized/Horses/38.jpg
/kaggle/input/vrpbpart2/resized/Horses/42.jpg
/kaggle/input/vrpbpart2/resized/Horses/33.jpg
/kaggle/input/vrpbpart2/resized/Horses/10.jpg
/kaggle/input/vrpbpart2/resized/Horses/54.jpg
/kaggle/input/vrpbpart2/resized/Horse

In [37]:
import cv2
import os
import numpy as np
import pickle
from os import listdir, makedirs
from os.path import isfile, join, exists, isdir
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split

In [38]:
cuda = cuda.is_available()
print(f'Train on gpu: {cuda}')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Train on gpu: True


In [39]:
rdir_path = '/kaggle/input/vrpbpart2/resized/'

In [40]:
# this function gets the image names according to their path as well as gives them a label according to the class.
def loadDataset():
    imgs = []
    labels = []
    label = -1
    
    dirnames = [f for f in os.listdir(rdir_path) if os.path.isdir(os.path.join(rdir_path, f))]
    
    for directory in dirnames:
        label = label + 1
        path = rdir_path+'/'+directory
        filenames = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        
        for file in filenames:
            imgPath = path+'/'+file
            imgs.append(imgPath)
            labels.append(label)
    
    return (imgs,labels)

In [41]:
# this function is used to get all the images of a particular class.
def getImgs(imgsPath):
    imgs = []
    for imgName in imgsPath:
        img = cv2.imread(imgName)
        imgs.append(img)
    imgs = np.asarray(imgs)
    return imgs

In [42]:
# load the dataset
dataset = loadDataset()

# train-test split the images.
train_ds,test_ds,train_labels,test_labels=train_test_split(
                                                    dataset[0],dataset[1],train_size=0.8,
                                                    random_state=77,shuffle=True,stratify=dataset[1])

train_ds = getImgs(train_ds)
test_ds = getImgs(test_ds)

In [43]:
# ## Loading images and labels
# (train_ds, train_labels), (test_ds, test_labels) = tfds.load(
#     "tf_flowers",
#     split=["train[:70%]", "train[:30%]"], ## Train test split
#     batch_size=-1,
#     as_supervised=True,  # Include labels
# )

# ## Resizing images
# train_ds = tf.image.resize(train_ds, (150, 150))
# test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels_enc = to_categorical(train_labels, num_classes=2)
test_labels_enc = to_categorical(test_labels, num_classes=2)

In [44]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [45]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 180, 280, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 280, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 280, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 140, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 140, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 140, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 70, 128)       0     

In [46]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [47]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels_enc, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
4/4 [==============================] - 27s 3s/step - loss: 2.2075 - accuracy: 0.8246 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
4/4 [==============================] - 1s 174ms/step - loss: 0.0133 - accuracy: 0.9912 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
4/4 [==============================] - 1s 184ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
4/4 [==============================] - 1s 173ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 1s 181ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 1s 195ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [8]:
print("Evaluate on test data")
results = model.evaluate(test_ds, test_labels_enc, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
35/35 [==============================] - 4s 114ms/step - loss: 0.1750 - accuracy: 0.9482
test loss, test acc: [0.17497128248214722, 0.9482288956642151]


In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
RF_model = RandomForestClassifier(n_estimators = 25, random_state = 42)

In [50]:
# Now we predict on the training set using the base model, 
# this time without adding the custom fully connected layers.
feature_extractor = base_model.predict(train_ds)
features = feature_extractor.reshape(feature_extractor.shape[0],-1)

X_for_RF = features

5/5 [==============================] - 4s 876ms/step


In [51]:
RF_model.fit(X_for_RF,train_labels_enc)

RandomForestClassifier(n_estimators=25, random_state=42)

In [52]:
# send test data through same feature extraction process.
feature_extractor_test = base_model.predict(test_ds)
test_features = feature_extractor_test.reshape(feature_extractor_test.shape[0],-1)

2/2 [==============================] - 1s 1s/step


In [53]:
preditction_RF = RF_model.predict(test_features)

In [54]:
from sklearn import metrics
print("Accuracy : ",metrics.accuracy_score(test_labels_enc,preditction_RF))

Accuracy :  1.0


In [55]:
from sklearn.svm import SVC

In [56]:
svclassifier = SVC(kernel = 'rbf',gamma = 'scale', random_state = 42)

In [57]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(train_labels)
# train_labels_svm = le.inverse_transform(train_labels)

In [58]:
# we do the same thing that we did for random forests.
X_for_SVM = features
svclassifier.fit(X_for_SVM,train_labels)

SVC(random_state=42)

In [59]:
svm_preds = svclassifier.predict(test_features)

In [60]:
from sklearn import metrics
print("Accuracy : ",metrics.accuracy_score(test_labels,svm_preds))

Accuracy :  1.0
